In [2]:
from pycaret.regression import *
import pandas as pd
data = pd.read_csv('new_cleaned_data.csv')

In [3]:
model = load_model('O3_rf_model_iris')

Transformation Pipeline and Model Successfully Loaded


In [4]:
data['DATETIMEDATA'] = pd.to_datetime(data['DATETIMEDATA'])

now = pd.Timestamp.now()
start_date = now.date()
end_date = start_date + pd.DateOffset(days=7)
next_week = pd.date_range(start=start_date, end=end_date, freq='D')
next_week_data_O3 = pd.DataFrame({'DATETIMEDATA': next_week})

next_week_data_O3['PM25'] = 0
next_week_data_O3['WS'] = 0
next_week_data_O3['TEMP'] = 0
next_week_data_O3['RH'] = 0
next_week_data_O3['WD'] = 0

predictions_o3 = predict_model(model, data=next_week_data_O3)
predictions_o3 = predictions_o3.rename(columns={'Label': 'O3'})
predictions_o3.rename(columns={'prediction_label': 'O3'}, inplace=True)
predictions_o3.to_csv('predictions_o3.csv')

In [5]:
model_1 = load_model('PM_et_model_iris')

Transformation Pipeline and Model Successfully Loaded


In [6]:
next_week_data_PM = pd.DataFrame({'DATETIMEDATA': next_week})

next_week_data_PM['O3'] = 0
next_week_data_PM['WS'] = 0
next_week_data_PM['TEMP'] = 0
next_week_data_PM['RH'] = 0
next_week_data_PM['WD'] = 0

predictions_pm = predict_model(model_1, data=next_week_data_PM)
predictions_pm = predictions_pm.rename(columns={'Label': 'PM25'})
predictions_pm.rename(columns={'prediction_label': 'PM25'}, inplace=True)
predictions_pm.to_csv('predictions_pm.csv')

In [15]:
prediction = pd.merge(predictions_o3, predictions_pm, on='DATETIMEDATA', how='outer')
prediction = prediction.drop(columns=['PM25_x','WS_x','TEMP_x','RH_x','WD_x','O3_y','WS_y','TEMP_y','RH_y','WD_y'])
prediction.rename(columns={'O3_x': 'O3'}, inplace=True)
prediction.rename(columns={'PM25_y': 'PM25'}, inplace=True)
prediction[['O3', 'PM25']] = prediction[['O3', 'PM25']].round(3)

In [16]:
prediction.to_csv('prediction.csv', index=False)